# Run JWST pipeline to reprocess NIRSpec MOS data

DRAFT demo notebook running the JWST pipeline on NIRSpec MOS data from ERO program 2736: SMACS0723 (JWST's "First Deep Field")

We make two key adjustments to input parameters:
* outlier rejection of cosmic rays and snowballs: detector1 jump step 
* extraction rows for 1D spectrum (x1d) from 2D spectrum (s2d): extract1d [ystart – ystop]

## This is a test

So far the final outputs aren't as good as those from MAST

### Pipeline Stages
uncal (counts) –> rate (slopes) –> cal (calibrated) –> s2d (2D spectra) –> x1d (1D spectra)

In [1]:
import os
home = os.path.expanduser("~")
from glob import glob
import numpy as np
from astropy.io import fits

In [2]:
# Define CRDS paths *before* importing jwst pipeline
# otherwise you'll get FileNotFoundError: [Errno 2] : No such file or directory: '$HOME/crds_cache/config/jwst/server_config'

#os.environ["CRDS_PATH"] = "$HOME/crds_cache"
#os.environ["CRDS_PATH"] = '/grp/crds/cache/' at STScI
os.environ["CRDS_PATH"] = os.path.join(home, "crds_cache")
os.makedirs(os.environ['CRDS_PATH'], exist_ok=True)

os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"
print(os.environ['CRDS_PATH'])

/Users/dcoe/crds_cache


In [3]:
import jwst
import crds
from jwst import datamodels
from jwst.pipeline import Detector1Pipeline   #calwebb_detector1
from jwst.pipeline import Spec2Pipeline       #calwebb_spec2
from jwst.pipeline import Spec3Pipeline       #calwebb_spec3
from jwst.extract_1d import Extract1dStep     #Extract1D Individual Step
import json

# JWST version won't be right if development version (may say 1.12.3 when it's really 1.12.6)
print("JWST Calibration Pipeline Version={}".format(jwst.__version__)) 
print("Current Operational CRDS Context = {}".format(crds.get_default_context()))

JWST Calibration Pipeline Version=1.12.5
Current Operational CRDS Context = jwst_1185.pmap


In [25]:
from IPython.display import JSON

In [4]:
import multiprocessing as mp  # to run faster on multiple cores in parallel
print("Number of processors: ", mp.cpu_count())

Number of processors:  10


In [5]:
import matplotlib # as mpl
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'font.size': 14})  # 18

from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
import matplotlib.patheffects as pe # https://stackoverflow.com/questions/25426599/matplotlib-how-to-buffer-label-text

from astropy.visualization import simple_norm, ImageNormalize, AsinhStretch, ManualInterval, LogStretch, LinearStretch
from astropy.stats import sigma_clip, sigma_clipped_stats  # , SigmaClip

In [6]:
cmap = 'viridis'
#bad_color = 1, 0.7, 0.7
bad_color = 'r'
cmap = matplotlib.colormaps[cmap]
cmap.set_bad(bad_color, 1.)

# Stage 1: Detector1Pipeline: uncal –> rate

In [68]:
output_dir = 'reprocess'
os.makedirs(output_dir, exist_ok=True)

In [69]:
params_det1 = {}
params_det1['jump'] = {  # https://jwst-pipeline.readthedocs.io/en/latest/jwst/jump/arguments.html
    'expand_large_events': True,
    'expand_factor':  3.0,  # default 2
    'min_sat_area':  15.0,  # default 1
    'min_jump_area': 15.0,  # default 5
    'maximum_cores': 'half',  # integer, 'quarter', 'half', 'all' -- run N times faster using N multiprocessors
}
params_det1['ramp_fit'] = {   # https://jwst-pipeline.readthedocs.io/en/latest/jwst/ramp_fitting/arguments.html
    'maximum_cores': 'half',  # integer, 'quarter', 'half', 'all' -- run N times faster using N multiprocessors
}

params_det1

{'jump': {'expand_large_events': True,
  'expand_factor': 3.0,
  'min_sat_area': 15.0,
  'min_jump_area': 15.0,
  'maximum_cores': 'half'},
 'ramp_fit': {'maximum_cores': 'half'}}

CEERS NIRSpec reduction parameters to improve jump step rejection of cosmic rays and snowballs  
https://web.corral.tacc.utexas.edu/ceersdata/DR07/NIRSpec/README_NIRSpec_DR0.7.txt

In [70]:
for uncal_file in sorted(uncal_files): 
    print('*' * 77)
    print(uncal_file)
    #print(os.path.basename(uncal_file))
    
    det1_result = Detector1Pipeline.call(
        uncal_file,
        save_results = True,
        output_dir = output_dir,
        steps = params_det1)

*****************************************************************************
data/jw02736007001_03101_00001_nrs1_uncal.fits


2024-01-23 12:04:08,094 - stpipe - INFO - PARS-JUMPSTEP parameters found: /Users/dcoe/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-jumpstep_0002.asdf
2024-01-23 12:04:08,106 - stpipe - INFO - PARS-DETECTOR1PIPELINE parameters found: /Users/dcoe/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-detector1pipeline_0004.asdf
2024-01-23 12:04:08,121 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2024-01-23 12:04:08,123 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2024-01-23 12:04:08,123 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2024-01-23 12:04:08,125 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2024-01-23 12:04:08,125 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2024-01-23 12:04:08,127 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2024-01-23 12:04:08,128 - stpipe.Detector1Pipeline.refpix - INFO - RefPi

*****************************************************************************
data/jw02736007001_03101_00001_nrs2_uncal.fits


2024-01-23 12:05:54,730 - stpipe - INFO - PARS-JUMPSTEP parameters found: /Users/dcoe/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-jumpstep_0002.asdf
2024-01-23 12:05:54,742 - stpipe - INFO - PARS-DETECTOR1PIPELINE parameters found: /Users/dcoe/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-detector1pipeline_0004.asdf
2024-01-23 12:05:54,758 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2024-01-23 12:05:54,759 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2024-01-23 12:05:54,760 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2024-01-23 12:05:54,761 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2024-01-23 12:05:54,761 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2024-01-23 12:05:54,762 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2024-01-23 12:05:54,764 - stpipe.Detector1Pipeline.refpix - INFO - RefPi

*****************************************************************************
data/jw02736007001_03103_00001_nrs1_uncal.fits


...

2024-01-23 12:53:54,384 - stpipe - INFO - PARS-JUMPSTEP parameters found: /Users/dcoe/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-jumpstep_0002.asdf
2024-01-23 12:53:54,395 - stpipe - INFO - PARS-DETECTOR1PIPELINE parameters found: /Users/dcoe/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-detector1pipeline_0004.asdf
2024-01-23 12:53:54,409 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2024-01-23 12:53:54,410 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2024-01-23 12:53:54,411 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2024-01-23 12:53:54,412 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2024-01-23 12:53:54,413 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2024-01-23 12:53:54,414 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2024-01-23 12:53:54,414 - stpipe.Detector1Pipeline.refpix - INFO - RefPi

*****************************************************************************
data/jw02736008001_03103_00003_nrs1_uncal.fits


2024-01-23 12:58:15,472 - stpipe - INFO - PARS-JUMPSTEP parameters found: /Users/dcoe/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-jumpstep_0002.asdf
2024-01-23 12:58:15,483 - stpipe - INFO - PARS-DETECTOR1PIPELINE parameters found: /Users/dcoe/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-detector1pipeline_0004.asdf
2024-01-23 12:58:15,497 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2024-01-23 12:58:15,499 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2024-01-23 12:58:15,499 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2024-01-23 12:58:15,500 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2024-01-23 12:58:15,502 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2024-01-23 12:58:15,503 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2024-01-23 12:58:15,503 - stpipe.Detector1Pipeline.refpix - INFO - RefPi

*****************************************************************************
data/jw02736008001_03103_00003_nrs2_uncal.fits


2024-01-23 13:03:22,826 - stpipe - INFO - PARS-JUMPSTEP parameters found: /Users/dcoe/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-jumpstep_0002.asdf
2024-01-23 13:03:22,836 - stpipe - INFO - PARS-DETECTOR1PIPELINE parameters found: /Users/dcoe/crds_cache/references/jwst/nirspec/jwst_nirspec_pars-detector1pipeline_0004.asdf
2024-01-23 13:03:22,853 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2024-01-23 13:03:22,854 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2024-01-23 13:03:22,855 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2024-01-23 13:03:22,856 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2024-01-23 13:03:22,856 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2024-01-23 13:03:22,857 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2024-01-23 13:03:22,858 - stpipe.Detector1Pipeline.refpix - INFO - RefPi

# Stage 2: Spec2Pipeline: rate –> cal, s2d, x1d

https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec2.html

Run on association files to perform nod-subtraction on standard 3-slitlet data.  
Each spec2 output will have 1 positive trace and 2 negative traces from the 3 exposures.  
The positive traces (3 exposures) will be added (drizzle combined) later below by Stage 3 Spec3Pipeline.

Spec2Pipeline takes 5 minutes on a MacBook Pro laptop (with reference files already downloaded)

In [75]:
asn_dir = 'associations'

In [ ]:
# rate files need to be in same directory as association file  

In [87]:
#spec2_asn_files = glob(os.path.join(asn_dir, '*.json'))
spec2_asn_files = glob(os.path.join(output_dir, '*.json'))
spec2_asn_files = select_files(spec2_asn_files, ['o007', 'spec2'])
spec2_asn_files = sorted(spec2_asn_files)
spec2_asn_files

['reprocess/jw02736-o007_20231116t144653_spec2_00001_asn.json',
 'reprocess/jw02736-o007_20231116t144653_spec2_00002_asn.json',
 'reprocess/jw02736-o007_20231116t144653_spec2_00003_asn.json',
 'reprocess/jw02736-o007_20231116t144653_spec2_00004_asn.json',
 'reprocess/jw02736-o007_20231116t144653_spec2_00005_asn.json',
 'reprocess/jw02736-o007_20231116t144653_spec2_00006_asn.json']

In [83]:
for asn_file in spec2_asn_files:
    print('-' * 33)
    print(os.path.basename(asn_file))
    with open(asn_file, 'r') as f_obj:
        asn_file_data = json.load(f_obj)

    for member in asn_file_data['products'][0]['members']:
        print(member['expname'], member['exptype'])

---------------------------------
jw02736-o007_20231116t144653_spec2_00001_asn.json
jw02736007001_03103_00003_nrs2_rate.fits science
jw02736007001_03103_00001_nrs2_rate.fits background
jw02736007001_03103_00002_nrs2_rate.fits background
---------------------------------
jw02736-o007_20231116t144653_spec2_00002_asn.json
jw02736007001_03103_00002_nrs2_rate.fits science
jw02736007001_03103_00001_nrs2_rate.fits background
jw02736007001_03103_00003_nrs2_rate.fits background
---------------------------------
jw02736-o007_20231116t144653_spec2_00003_asn.json
jw02736007001_03103_00001_nrs2_rate.fits science
jw02736007001_03103_00002_nrs2_rate.fits background
jw02736007001_03103_00003_nrs2_rate.fits background
---------------------------------
jw02736-o007_20231116t144653_spec2_00004_asn.json
jw02736007001_03103_00003_nrs1_rate.fits science
jw02736007001_03103_00001_nrs1_rate.fits background
jw02736007001_03103_00002_nrs1_rate.fits background
---------------------------------
jw02736-o007_20231

In [84]:
output_dir = 'reprocess'

In [85]:
params_spec2 = {}
params_spec2['resample_spec'] = {  
    'weight_type': 'exptime'
}

params_spec2

{'resample_spec': {'weight_type': 'exptime'}}

In [ ]:
for asn_file in spec2_asn_files:
    spec2_result = Spec2Pipeline.call(
        asn_file,
        save_results = True,
        output_dir = output_dir,
        steps = params_spec2
    )

2024-01-23 13:45:45,886 - CRDS - ERROR -  Error determining best reference for 'pars-pixelreplacestep'  =   Unknown reference type 'pars-pixelreplacestep'
2024-01-23 13:45:45,899 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2024-01-23 13:45:45,900 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2024-01-23 13:45:45,901 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2024-01-23 13:45:45,902 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2024-01-23 13:45:45,903 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2024-01-23 13:45:45,904 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2024-01-23 13:45:45,906 - stpipe.Spec2Pipeline.master_background_mos - INFO - MasterBackgroundMosStep instance created.
2024-01-23 13:45:45,907 - stpipe.Spec2Pipeline.master_background_mos.flat_field - INFO - FlatFieldStep instance created.
2024-01-23

In [ ]:
rate_files = glob(os.path.join(output_dir, '*_rate.fits'))

for rate_file in rate_files:
    # Download MSA metafile
    msa_file = fits.getval(rate_file, 'MSAMETFL')
    download_jwst_files([msa_file], output_dir)

Downloading URL https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:jwst/product/jw02736007001_01_msa.fits to reprocess/jw02736007001_01_msa.fits ... [Done]

# Stage 3: Spec3Pipeline: combining exposures

https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec3.html

In [96]:
asn_files = glob(os.path.join(output_dir, '*.json'))
spec3_asn_file = select_files(asn_files, ['o007', 'spec3'])
spec3_asn_file

'reprocess/jw02736-o007_20231116t144653_spec3_00001_asn.json'

In [97]:
with open(spec3_asn_file) as f_obj:
    asn_data = json.load(f_obj)
    spec2_cal_files = []
    for member in asn_data['products'][0]['members']:
        if member['exptype'] == 'science':
            spec2_cal_file = member['expname']
            spec2_cal_files.append(spec2_cal_file)
            print(spec2_cal_file)

jw02736007001_03103_00001_nrs1_cal.fits
jw02736007001_03103_00001_nrs2_cal.fits
jw02736007001_03103_00002_nrs1_cal.fits
jw02736007001_03103_00002_nrs2_cal.fits
jw02736007001_03103_00003_nrs1_cal.fits
jw02736007001_03103_00003_nrs2_cal.fits


In [102]:
# Create a subdirectory for the many output files (several for each extracted source)
spec3_output_dir = os.path.join(output_dir, 'spec3')
os.makedirs(spec3_output_dir, exist_ok=True)
spec3_output_dir

'reprocess/spec3'

https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec3.html

In [103]:
params_spec3 = {}
# I tried this both ways:
params_spec3['outlier_detection'] = {'skip' : True} # too aggressive! Flag outlier bad pixels and cosmic rays in DQ array of each input image
#params_spec3['outlier_detection'] = {'skip' : False} # too aggressive? Flag outlier bad pixels and cosmic rays in DQ array of each input image

params_spec3['resample_spec']     = {'weight_type' : 'exptime'}  # single value exposure time (vs. pixel-based 'ivm' inverse read noise)
params_spec3['extract_1d']        = {'use_source_posn' : 'True'} # extraction rows based on expected source position
params_spec3

{'outlier_detection': {'skip': False},
 'resample_spec': {'weight_type': 'exptime'},
 'extract_1d': {'use_source_posn': 'True'}}

In [104]:
spec3_result = Spec3Pipeline.call(
    spec3_asn_file,
    save_results = True,
    output_dir = spec3_output_dir,
    steps = params_spec3
)

2024-01-25 07:39:39,939 - CRDS - ERROR -  Error determining best reference for 'pars-outlierdetectionstep'  =    parameter='META.EXPOSURE.TYPE [EXP_TYPE]' value='NRS_MSASPEC' is not in ['NRS_IFU']
2024-01-25 07:39:40,409 - stpipe.Spec3Pipeline - INFO - Spec3Pipeline instance created.
2024-01-25 07:39:40,410 - stpipe.Spec3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2024-01-25 07:39:40,410 - stpipe.Spec3Pipeline.master_background - INFO - MasterBackgroundStep instance created.
2024-01-25 07:39:40,411 - stpipe.Spec3Pipeline.mrs_imatch - INFO - MRSIMatchStep instance created.
2024-01-25 07:39:40,412 - stpipe.Spec3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2024-01-25 07:39:40,414 - stpipe.Spec3Pipeline.resample_spec - INFO - ResampleSpecStep instance created.
2024-01-25 07:39:40,415 - stpipe.Spec3Pipeline.cube_build - INFO - CubeBuildStep instance created.
2024-01-25 07:39:40,417 - stpipe.Spec3Pipeline.extract_1d - INFO - Extract1dStep in